$ \newcommand{\bra}[1]{\langle #1|} $
$ \newcommand{\ket}[1]{|#1\rangle} $
$ \newcommand{\braket}[2]{\langle #1|#2\rangle} $
$ \newcommand{\ketbra}[2]{| #1\rangle \langle #2|} $
$ \newcommand{\tr}{{\rm tr}} $
$ \newcommand{\i}{{\color{blue} i}} $
$ \newcommand{\Hil}{{\cal H}} $
$ \newcommand{\V}{{\cal V}} $
$ \newcommand{\bn}{{\bf n}} $
   
# 1. Introduction to qubit physics

To understand how quantum gates are implemented, we first need to examine the qubit's physical system. In this notebook, we will see how a qubit is encoded in superconducting circuits, which approximations can be used depending on our goals, what effects arise from the interactions with the thermal bath, how the qubit-resonator interaction enables measurements, and how to define the parameters of this systems in OpenPulse.

## 1.1. The isolated qubit system

A **qubit** is any system that can be escribed by a two-dimensional Hilbert space, $\mathbb{C}^2$. Usually, we denote by $\{\ket{0},\ket{1}\}$ an orthonormal basis, so any arbitrary **qubit state** can be described as:
$$\ket{\psi}=\alpha\ket{0}+\beta\ket{1};\quad\alpha,\beta\in\mathbb{C},\ |\alpha|^2+|\beta|^2=1. $$
From here, we can derive all the arithmetic used in quantum computing, but the focus of this course will be on how to encode this space in a real physical system and how to implement the unitary gates used in quantum circuits. Specifically, we'll see how the qubits are encoded it transmons, also called charge-qubits.

#### Encoding a qubit in superconducting circuits
The superconducting LC circuit is a basic case where we can define a qubit, this circuit consists on a capacitor and an inductor that exchange their electric and magnetic energies in the same way as a harmonic oscillator. In fact, we can describe its energy as a quantum harmonic oscillator (QHO):
$$H_{LC}=\hbar\omega_q\left(a^\dagger a+\frac{1}{2}\right),$$
where $\omega_q$ is called the **resonance frequency**. As it's well known, the QHO has an infinite number of energy eigenstates $H_{LC}\ket{n}=E_n\ket{n}=\hbar\omega_q\left(n+1/2\right)$.<br>
<br> 
A qubit encoding can be performed by restricting the system to a subspace spanned by two of these eigenstates, for example, the conveniently labeled $\{\ket{0},\ket{1}\}$, which correspond to the ground and first excited states of the oscillator. Since energy is an observable, in principle we could measure it, collapsing the system to either $\ket{0}$ or $\ket{1}$ for any superposition of these states. We could also induce transitions between them by applying energy pulses (photons) with the appropriate energy given by the resonance frequency. But there is a major problem with the LC circuit: its harmonicity. <br>
<br>
Imagine we want to transition from the state $\ket{0}$ to $\ket{1}$. To do so, we need to apply a pulse of frequency $\omega_q$, but this frequency is also the transition frequency for $\ket{1}\leftrightarrow\ket{2}$, $\ket{2}\leftrightarrow\ket{3}$, and so on, due to the harmonicity of the oscillator. This means that we do not have control over which transition will occur. For example, if the system is initially in the state $\ket{1}$ and we want to apply an $X$ gate to transition to $\ket{0}$, the result of applying a pulse of frequency $\omega_q$ may be the desired transition to $\ket{0}$ (via stimulated emission), but also the undesired transition to $\ket{2}$ (via absorption). The latter leads to population outside the computational subspace, resulting in loss of coherence. <br>
<br>
The most popular configuration for superconducting qubits, including the so-called **transmons**, is achieved by substituting the inductor with a Josephson junction. This element changes the potential energy profile of the oscillator, resulting in non-equidistant energy levels: 
$$\left(E_{n+1}-E_{n}\right)<\left(E_{n}-E_{n-1}\right).$$
In this case, the oscillator transitions have different frequencies, allowing control over which transitions can occur when applying a pulse. Considering the most generic expression for its Hamiltonian:
$$H=\sum_{n=0}^\infty\hbar\omega_{n,n+1}a^\dagger a,\quad \omega_{n,n+1}<\omega_{n-1,n}\quad \forall n;$$
where $\omega_{n,n+1}$ is the $\ket{n}\leftrightarrow\ket{n+1}$ transition frequency and the ground-state energy is set to $E_0=0$. If we want to encode our qubit in the $\{\ket{0},\ket{1}\}$ subspace, we need to use pulses with frequency $\omega_{01}$, avoiding any other undesired transition.<br>
<br>
Usually, we will consider only the computational subspace of the system, assuming negligible populations in any other excited states, using the same model as the LC circuit. Rearranging the energy offset, we can express the Hamiltonian in terms of Pauli operators:
$$H=-\hbar\omega_q\frac{\sigma_z}{2},$$
with $\omega_q=\omega_{01}$.<br>
<br>
Nevertheless, it could be interesting to study the effects of higher excited states. The transmon system is usually modeled as a Duffing oscillator:
$$H=\hbar\omega_qa^\dagger a+\frac{\hbar\alpha}{2}a^\dagger a^\dagger a a,$$
where $\omega_q=\omega_{01}$ is the resonance frequency and $\alpha=\omega_{12}-\omega_{01}<0$ is the **anharmonicity** of the qubit. 

#### The qubit rotating frame

Before studying the dynamics of the pulse interaction with the qubit, we need to consider the intrinsic dynamics of the ocillator where we have encode it. Let's assume a 2-level qubit system initially prepared in the state: $$\ket{\psi_0}=\cos(\theta/2)\ket{0}+e^{i\phi}\sin(\theta/2)\ket{1}.$$
In the absence of external interactions, the time evolution given by the Schrödinger's equation leads, in this case, to the following time evolution unitary operator:
$$ U(t)=\exp\left(i \frac{\omega_q t}{2}\sigma_z\right) = \cos\left(\omega_q t/2\right)I+i\sin\left(\omega_q t/2\right)\sigma_z.$$
The time evolution for the qubit state is, omitting global phases:
$$\ket{\psi(t)}=\cos\left(\theta/2\right)\ket{0}+e^{i\left(\phi-\omega_q t\right)}\sin\left(\theta/2\right)\ket{1}\equiv \text{RZ}\left(-\omega_q t\right),$$
which corresponds to a continous RZ rotation on the Bloch sphere, with a predictable rate set by its resonance frequency. <br>
<br>
There are two main reasons why keeping track of this intrinsic time evolution is important:

- It's often convenient to work in the **rotation frame** of the qubit, where this evolution is removed by applying the known $U(t)^\dagger$. In this frame, pulses have more simpler expressions.
- **RZ gates** can be implemented by just letting the system evolve, without the need of applying any pulse. This will be later explored for the virtual RZ gates.

## 1.2. The open qubit system

The models from before assumed that the qubit is an isolated system, meaning that there are no interactions with its environment, so its energy remains constant. This is far from reality, as there are very relevant effects that are expected for an oscillator in a thermal bath. In superconducting computers, the qubits are immersed in a very cold thermal bath, on the order of tens to hundreds of mK, achieved by various cryogenic systems. The objective is to reduce the probability that a residual photon in the cavity with energy $\hbar\omega_q$ interacts with the qubit. Defining $k_BT$ as the mean energy of the interactions in a photonic bath at temperature $T$, the regime where the isolated approximation is valid is when $k_BT\ll \hbar\omega_q$. Nevertheless, there will always be a residual population for any excited states, given by the Gibbs thermal state:
$$P_n=\frac{1}{Z}\exp\left(-\frac{\hbar\omega_n}{k_BT}\right),$$
where $\hbar\omega_n$ is the energy of the eigenstate $\ket{n}$ and $Z$ is the partition function given by:
$$Z=\sum_{n=0}^\infty \exp\left(-\frac{\hbar\omega_n}{k_BT}\right).$$


In other words, the cold thermal bath will bring the qubit to some state very close to $\ket{0}$, with residual excited populations, from which we can operate assuming that we are working with an isolated qubit, applying the pulses to perform the desired gates in our computations. This is why the computational state $\ket{0}$ was conveniently chosen to be the ground-state and also the default initialization in many quantum circuit formalisms.<br>
<br>
Some interesting questions may arise: How does the qubit system evolve during this cooling? Does the thermal bath have a relevant effect on the qubit during the computations?<br>
This cooling process is called **thermalization** and it can be modeled using the Lindblad equations. This method consists of modeling the possible interactions of the bath's photons with the qubit, dividing them in three types:

- **Excitation**: a photon with energy $\hbar\omega_q$ interacts with the qubit, increasing its energy.
- **Relaxation**: the qubit spontaneously emits a photon of energy $\hbar\omega_q$ to the bath, decreasing its energy.
- **Dephasing**: the energy of the qubit remains the same, but the relative phase of its state shifts.

Each of this interactions has a probability of happening that depends on the bath's temperature, for example, a very warm bath will have a greater excitation probability, as its photons are more energetic. These probabilities can be parametrized by the **relaxation times**:

- $T_1$: the **longitudinal** relaxation time, which is related to the balance between excitation and relaxation interactions.
- $T_2$: the **tranverse** relaxation time, which includes the effect of the dephasing interactions.

For a system in an arbitrary mixed state:
$$\rho(0)=\begin{pmatrix}
\rho_{00}(0) & \rho_{01}(0)\\
\rho_{10}(0) & \rho_{11}(0)
\end{pmatrix},$$
the differential equation derived from the Lindblad equations has the following solution for the qubit state at any time $t$:
$$\rho(t)=\begin{pmatrix}
\rho_{00}^{eq} +\Bigl(\rho_{00}(0)-\rho_{00}^{eq}\Bigl)e^{-t/T_1} & \rho_{01}(0)e^{-t/T_2}e^{i\omega_q t}\\
\rho_{10}(0)e^{-t/T_2}e^{-i\omega_q t} & \rho_{11}^{eq} +\Bigl(\rho_{11}(0)-\rho_{11}^{eq}\Bigl)e^{-t/T_1}
\end{pmatrix},$$
where $\rho^{(eq)}$ also depends on the bath's temperature. In fact, if we take the time limit:
$$\rho^{(eq)}=\lim_{t\rightarrow\infty}\rho(t)=\begin{pmatrix}
\rho_{00}^{eq} &0\\
0 & \rho_{11}^{eq}
\end{pmatrix},$$
that corresponds to the Gibbs state from before:
$$
\begin{align*}
\rho_{00}^{eq} =  \frac{1}{Z}\exp\left(-\frac{\hbar\omega_0}{k_BT}\right),\quad
\rho_{11}^{eq} =  \frac{1}{Z}\exp\left(-\frac{\hbar\omega_1}{k_BT}\right).
\end{align*}
$$
This means that, for an arbitrary initial state, the qubit will decay to the Gibbs thermal state in some time parametrized by the relaxation time $T_1$ and $T_2$. This decay not only affecs its populations, it also changes its phase in an spiral-like form. Here there is an illustrative video of the process, for the initial state $\ket{+}$:

<br>
<br>
<div style="text-align: center;">
  <video width="720" height="480" controls>
    <source src="./media/BlochDecayScene.mp4" type="video/mp4">
    Tu navegador no soporta el video.
  </video>
</div>
<br>
<br>

As you can see, the state of the qubit (red dot) evolves in an spiraling decay (yellow curve) to some mixed state near $\ket{0}$, for a cold thermal bath. The blue sphere represents the norm of the state in the Bloch sphere, that is, its purity. At first, the sudden loss of purity would mean a loss of coherence if it happens during a computation, as we have lost track of the qubit's state. <br>
<br>
On the other hand, after enough time, the qubit state purity increases again until it reaches some asymptotic limit on the Gibbs state. This can be understood as the **initialization** of the qubit after a computation. In other words, the thermalization of the qubit let's us initialized a hopefully almost pure $\ket{0}$ state from where the computations can be performed with coherence, but it also limits the time on which operations can be performed, as it will keep interacting with the qubit, changing its state in a non-unitary fashion. This operational time, called **coherence time** is characterized by the relaxation times from before, and limits the amount of time, and so the number of operations, we can operate on the qubit.

## 1.3. The qubit-resonator coupling

## 1.4. The qubit frame in OpenPulse